In [1]:
import os
import math
import shutil
import numpy as np
from multiprocessing import Pool
from keras.optimizers import *
from keras.losses import categorical_crossentropy
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from random import seed

Using TensorFlow backend.


In [2]:
GPU = '0'
RND = 777
RUN = 'D'
OUT_DIR = 'out'
TENSORBOARD_DIR = '/tensorboard/tf-speech-v2/%s_$fold$' % RUN
MODELS_DIR = '%s/models/run_%s/fold_$fold$' % (OUT_DIR, RUN)
INPUT_SIZE = (96, 96, 1)  # n_mels x width x 1ch
FOLDS = 10

In [3]:
np.random.seed(RND)
seed(RND)

In [4]:
# make only specific GPU to be utilized
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = GPU

In [5]:
%run '../data-generator.ipynb'
%run '../models.ipynb'

In [6]:
def choose_batch(n, train_X, train_Y, train_files, val_files):
    assert isinstance(val_files, set)

    # extra random indexes to search for files not in val_files
    def _extra_indexes():
        return np.random.randint(0, len(train_X), size=int(n * 0.15))

    ii = np.random.randint(0, len(train_X), size=n)
    extra_ii = []

    replaced = 0

    # replace indexes with files occuring in val_files
    for j in range(len(ii)):
        if '(silence)' != train_files[ii[j]]:
            while train_files[ii[j]] in val_files:
                if len(extra_ii) == 0: extra_ii = _extra_indexes()
                ii[j], extra_ii = extra_ii[0], extra_ii[1:]
                replaced += 1

    X = train_X[ii]
    Y = train_Y[ii]
    files = train_files[ii]
    
    return X, Y, files

In [7]:
train_X = np.memmap('%s/train_X.mem' % OUT_DIR, np.float32,
                    'r').reshape((-1, ) + INPUT_SIZE)
train_Y = np.memmap('%s/train_Y.mem' % OUT_DIR, np.float32, 'r').reshape(
    (-1, len(LABELS)))

train_files = np.load('%s/train_files.npy' % OUT_DIR)

assert len(train_Y) == len(train_X)
assert len(train_files) == len(train_X)

print('len(train_X):', len(train_X))

len(train_X): 1000000


In [8]:
# training params
N_PER_BATCH = 500
# last number splits train set into XX epochs
STEPS_PER_EPOCH = len(train_X) // N_PER_BATCH // 10
N_EPOCHS = 100

In [ ]:
test_X = np.memmap('%s/test/test_X.mem' % (OUT_DIR), np.float32,
                   'r').reshape((-1, ) + INPUT_SIZE)

In [ ]:
for fold in range(0, FOLDS // 2):

    print('fold:', fold)

    # read val data
    val_X = np.load('%s/val/val_X_%d.npy' % (OUT_DIR, fold))
    val_Y = np.load('%s/val/val_Y_%d.npy' % (OUT_DIR, fold))
    val_files = np.load('%s/val/val_files_%d.npy' % (OUT_DIR, fold))
    assert len(val_X) == len(val_files)
    assert len(val_Y) == len(val_files)
    print('len(val_X):', len(val_X))
    val_files = set(val_files)

    # create dir to store models
    models_dir = MODELS_DIR.replace('$fold$', str(fold))
    os.makedirs(models_dir, exist_ok=True)
    print('models_dir:', models_dir)

    def train_generator(n_per_batch):
        while True:
            X, Y, files = choose_batch(n_per_batch, train_X, train_Y,
                                       train_files, val_files)
            yield (X, Y)

    # rm/create tensorboard dir
    tensorboard_dir = TENSORBOARD_DIR.replace('$fold$', str(fold))
    shutil.rmtree(tensorboard_dir, ignore_errors=True)
    os.makedirs(tensorboard_dir)
    print('tensorboard_dir:', tensorboard_dir)

    # create model
    model = Model_4(input_size=INPUT_SIZE, output_size=len(LABELS))
    model.build()
    optimizer = RMSprop(lr=1e-3)
    model.m.compile(
        optimizer=optimizer, loss=categorical_crossentropy, metrics=['accuracy']\
    )

    model.m.fit_generator(
        train_generator(N_PER_BATCH),
        STEPS_PER_EPOCH,
        epochs=N_EPOCHS,
        validation_data=(val_X, val_Y),
        callbacks=[
            TensorBoard(log_dir=tensorboard_dir),
            ModelCheckpoint(
                models_dir +
                '/e{epoch:03d}-l={loss:.5f}-vl={val_loss:.5f}-a={acc:.5f}-va={val_acc:.5f}.h5',
                monitor='val_acc',
                verbose=0,
                save_best_only=True,
                save_weights_only=False,
                mode='auto'),
            ReduceLROnPlateau(
                monitor='loss', factor=0.2, patience=1, min_lr=1e-9,
                verbose=1),
            EarlyStopping(
                monitor='val_acc',
                min_delta=0.000001,
                patience=5,
                verbose=1,
                mode='auto')
        ])

    # predict on holdout
    holdout_X = np.load('%s/holdout/holdout_X.npy' % (OUT_DIR))
    holdout_Y = np.load('%s/holdout/holdout_Y.npy' % (OUT_DIR))
    hp = model.m.predict(holdout_X)
    np.save('%s/holdout/holdout_predictions_%d.npy' % (OUT_DIR, fold), hp)

    # eval on holdout
    print('evaluation on holdout:')
    print(model.m.metrics_names)
    print(model.m.evaluate(holdout_X, holdout_Y, verbose=0))

    # predict on test data
    test_predictions = model.m.predict(test_X, verbose=1, batch_size=100)
    np.save('%s/test/test_predictions_%d.npy' % (OUT_DIR, fold),
            test_predictions)

    print('')

fold: 0
len(val_X): 2441
models_dir: out/models/run_D/fold_0
tensorboard_dir: /tensorboard/tf-speech-v2/D_0
Epoch 1/100
200/200 [==============================] - 140s - loss: 1.3836 - acc: 0.5808 - val_loss: 1.5333 - val_acc: 0.6723
Epoch 2/100
200/200 [==============================] - 135s - loss: 0.5963 - acc: 0.8080 - val_loss: 0.4971 - val_acc: 0.9050
Epoch 3/100
 80/200 [===========>..................] - ETA: 80s - loss: 0.1041 - acc: 0.9707

In [ ]:
# from keras.models import load_model

# m1 = load_model('out_64x64_1m/models/run_B/fold_9/e012-l=0.02628-vl=0.38592-a=0.99213-va=0.94754.h5')

# m2 = load_model('out_64x64_1m/models/run_B/fold_1/e007-l=0.19546-vl=0.30110-a=0.93992-va=0.93200.h5')

# m3 = load_model('out_64x64_1m/models/run_B/fold_5/e013-l=0.02991-vl=0.36607-a=0.99127-va=0.95451.h5')

# for i in range(1000):
#     d = np.random.randn(1, 64, 64, 1)
#     l1 = LABELS[np.argmax(m1.predict(d))]
#     l2 = LABELS[np.argmax(m2.predict(d))]
#     l3 = LABELS[np.argmax(m3.predict(d))]
#     if l1 != 'silence' or l2 != 'silence' or l3 != 'silence': print(l1, l2, l3)